In [14]:
import requests
import pandas as pd

#### read excel

In [83]:
rename_dict = {
    'Город': 'city',
    'Населенный пункт': 'city',
    #
    'Адреса': 'address',
    'Адрес': 'address',
    #
    'Название': 'name'
}

In [84]:
#file = 'ЛПУ по городам 20.09.2022 (2) (1) (1).xlsx'
file = 'Лаборатории_по_городам_от_03_09_22_1.xlsx'

df = pd.read_excel(file)
df.rename(columns=rename_dict, inplace=True)

In [85]:
df.head()

,city,address,Название лаборатории
0,г. Москва,"город Москва, Шоссе Энтузиастов, дом 22/18",Инвитро
1,г. Москва,"город Москва, улица Солдатская, дом 10, корп. 2",Инвитро
2,г. Москва,"город Москва, 1-ый Кожуховский пр-д, дом 9",Инвитро
3,г. Москва,"город Москва, улица Профсоюзная, дом 5/9",Инвитро
4,г. Москва,"город Москва, улица Староалексеевская, дом 8",Инвитро


#### API

In [3]:
url = 'https://api.apidq.io/v1/clean/address'

In [71]:
data = {
  "query": "г. Севастополь, Республика Крым, город Севастополь, пр-т Октябрьской революции, дом 42",
  "countryCode": "RU"
}

In [72]:
r = requests.post(url, json=data)

In [73]:
true = True
false = False


In [74]:
d = eval(r.text)

In [80]:
d['region']['fullName']

'г Севастополь'

In [76]:
d['addressFull']

'г Севастополь, пр-кт Октябрьской революции, дом 42'

In [77]:
d['city']['fullName']

''

In [78]:
d['settlement']['fullName']

''

In [79]:
d

{'original': 'г. севастополь, республика крым, город севастополь, пр-т октябрьской революции, дом 42',
 'addressFull': 'г Севастополь, пр-кт Октябрьской революции, дом 42',
 'address': 'г Севастополь, пр-кт Октябрьской революции',
 'postcodeIn': '',
 'postcode': '299038',
 'region': {'fullName': 'г Севастополь',
  'name': 'Севастополь',
  'type': 'г',
  'codes': {'fias': '6fdecb78-893a-4e3f-a5ba-aa062459463b',
   'ga': 'RU0920000000000000000000000',
   'osm': 'R1574364',
   'gar': '1461214',
   'kladr': '9200000000000'}},
 'area': {'fullName': '',
  'name': '',
  'type': '',
  'codes': {'fias': '', 'ga': '', 'osm': '', 'gar': '', 'kladr': ''}},
 'city': {'fullName': '',
  'name': '',
  'type': '',
  'codes': {'fias': '', 'ga': '', 'osm': '', 'gar': '', 'kladr': ''}},
 'cityArea': {'fullName': '',
  'name': '',
  'type': '',
  'codes': {'fias': '', 'ga': '', 'osm': '', 'gar': '', 'kladr': ''}},
 'settlement': {'fullName': '',
  'name': '',
  'type': '',
  'codes': {'fias': '', 'ga': '',

#### clean address

In [86]:
true = True
false = False

df['address_clean'] = ''
df['city_clean'] = ''

big_cities = [
    'г Москва',
    'г Санкт-Петербург',
    'г Севастополь'
]

In [87]:
for i in range(df.shape[0]):
    temp_addr = ''
    addressFull = ''
    bigcity = ''
    city = ''
    settlement = ''
    
    if df['city'][i] not in df['address']:
        temp_addr = df['city'][i] + ', ' + df['address'][i]
    else:
        temp_addr = df['address'][i]
        
    data = {
      "query": temp_addr,
      "countryCode": "RU"
    }
    
    r = requests.post(url, json=data)
    
    d = eval(r.text)
    
    try:
        bigcity = d['region']['fullName']
    except:
        bigcity = ''
        
    try:
        addressFull = d['addressFull']
    except:
        addressFull = 'Не найдено'
    
    try:
        city = d['city']['fullName']
    except:
        addressFull = ''
    
    try:
        settlement = d['settlement']['fullName']
    except:
        settlement = 'Не найдено'
    
    df['address_clean'][i] = addressFull
    
    if bigcity in big_cities:
        df['city_clean'][i] = bigcity
    elif city:
        df['city_clean'][i] = city
    else:
        df['city_clean'][i] = settlement

In [88]:
#df.to_excel('test1.xlsx')

In [89]:
df

,city,address,Название лаборатории,address_clean,city_clean
0,г. Москва,"город Москва, Шоссе Энтузиастов, дом 22/18",Инвитро,"г Москва, Энтузиастов ш, дом 22/18",г Москва
1,г. Москва,"город Москва, улица Солдатская, дом 10, корп. 2",Инвитро,"г Москва, ул Солдатская, дом 10, корпус 2",г Москва
2,г. Москва,"город Москва, 1-ый Кожуховский пр-д, дом 9",Инвитро,"г Москва, 1-й Кожуховский проезд, дом 9",г Москва
3,г. Москва,"город Москва, улица Профсоюзная, дом 5/9",Инвитро,"г Москва, ул Профсоюзная, дом 5/9",г Москва
4,г. Москва,"город Москва, улица Староалексеевская, дом 8",Инвитро,"г Москва, ул Староалексеевская, дом 8",г Москва
...,...,...,...,...,...
3086,Уссурийск,"Чичерина, 83",Юнилаб,"Приморский край, г Уссурийск, ул Чичерина, дом 83",г Уссурийск
3087,Уссурийск,"Пионерская, 19",Юнилаб,"Приморский край, г Уссурийск, ул Пионерская, д...",г Уссурийск
3088,Фокино,"Клубная,15а",Юнилаб,"Приморский край, г Фокино, ул Клубная, дом 15а",г Фокино
3089,Ю Сахалинск,"Чехова, 87",Юнилаб,"Сахалинская обл, г Южно-Сахалинск, ул Чехова, ...",г Южно-Сахалинск


In [42]:
d

{'code': 5, 'message': 'Ничего не найдено', 'details': []}

In [90]:
df.to_excel('clean_' + file)